In [ ]:
import pandas as pd

df= pd.read_csv('winequality-red.csv', delimiter=';')

y = df['quality']
X = df.drop(columns=['quality'])
variables_predictoras = df[['volatile acidity', 'density', 'sulphates', 'alcohol', 'citric acid']]

In [ ]:
for col in variables_predictoras.columns:
    x_promedio = variables_predictoras[col].mean()
    y_promedio = y.mean()
    Sxx = ((variables_predictoras[col] - x_promedio) ** 2).sum()
    Sxy = ((variables_predictoras[col] - x_promedio) * (y - y_promedio)).sum()
    Syy = ((y - y_promedio) ** 2).sum()
    b1 = Sxy / Sxx
    b0 = y_promedio - b1 * x_promedio
